In [ ]:
!pip install unsloth xformers trl peft accelerate bitsandbytes faiss-cpu langchain-community langchain-huggingface sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.

# RAG

In [ ]:
import gdown

# Скачивание и загрузка клинических рекомендаций
guidelines_id = "1Ctvi5eS39zYDY5paXmIgV9PnVaiRq1UU"
guidelines_name = "russco"
gdown.download(id=guidelines_id, output="guidelines.zip", quiet=False)
!unzip -O CP866 -o -q guidelines.zip -d guidelines

Downloading...
From: https://drive.google.com/uc?id=1Ctvi5eS39zYDY5paXmIgV9PnVaiRq1UU
To: /content/guidelines.zip
100%|██████████| 994k/994k [00:00<00:00, 16.1MB/s]


In [ ]:
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Настройка заголовков для разделения
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

# Инициализируем сплиттеры
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Загрузка документов
loader = DirectoryLoader("./guidelines/"+guidelines_name, glob="./*.md", loader_cls=TextLoader)
documents = loader.load()

final_chunks = []
for doc in documents:
    file_source = doc.metadata.get('source', 'unknown')
    file_name = "".join(os.path.basename(file_source).split(".")[:-1])

    header_splits = markdown_splitter.split_text(doc.page_content)
    for header_chunk in header_splits:
        header_chunk.metadata['source'] = file_name
        sub_chunks = text_splitter.split_documents([header_chunk])
        final_chunks.extend(sub_chunks)

# Загрузка модели эмбеддингов
model_name = "intfloat/multilingual-e5-small"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# 3. Создание векторной базы
vector_db = FAISS.from_documents(final_chunks, embeddings)
vector_db.save_local("russco_faiss")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

# Fine-tuning

In [ ]:
import json
import random
import gdown

# Скачивание и загрузка датасета
dataset_id = "1PY_woE3FPjrZcVDKcShVmdCFyBpv_s60"
gdown.download(id=dataset_id, output="full_dset.json", quiet=False)

with open("./full_dset.json", "r") as file:
    data_list = json.load(file)

# Разделение датасета на выборки
random.seed(42)
random.shuffle(data_list)
n_test = round(0.1*len(data_list))
data_list_test, data_list_train = data_list[0:n_test], data_list[n_test:]

# Сохранение выборок
with open("./train_dset.json", 'w', encoding='utf-8') as json_file:
    json.dump(data_list_train, json_file, indent=4, ensure_ascii=False)
with open("./test_dset.json", 'w', encoding='utf-8') as json_file:
    json.dump(data_list_test, json_file, indent=4, ensure_ascii=False)

print(f"\n\nЧисло записей: {len(data_list)}\nРазмер обучающей выборки: {len(data_list_train)}\nРазмер тестовой выборки: {len(data_list_test)}\n")
for key, value in data_list[1].items():
  print(f"===== {key} =====\n{value}\n")

Downloading...
From: https://drive.google.com/uc?id=1PY_woE3FPjrZcVDKcShVmdCFyBpv_s60
To: /content/full_dset.json
100%|██████████| 280k/280k [00:00<00:00, 5.13MB/s]



Число записей: 218
Размер обучающей выборки: 196
Размер тестовой выборки: 22

===== treatment_plan =====
Диагноз: Рак правой молочной железы, стадия IIIC (pT2N3M0).
Анамнез: Операция на первом этапе. В 12 лимфоузлах метастазы (pN3).
Результаты анализов: HER2/neu: 3+; РЭ: 0.
План лечения:
Адъювантная терапия: 4 цикла АС -> 4 цикла таксанов в комбинации только с Трастузумабом.

===== clinical_guidelines =====
Назначение: Адъювантная терапия: 4 цикла АС -> 4 цикла таксанов в комбинации с Трастузумабом и Пертузумабом. При поражении лимфоузлов N2–3 рекомендуется добавление Пертузумаба к Трастузумабу на 12 месяцев.

===== corrections =====
Ошибка: Отсутствие Пертузумаба при массивном поражении лимфоузлов (N3).
Стадия pN3 является критерием высокого риска, требующим двойной анти-HER2 блокады.



In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048

# Загрузка языковой модели
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
)

# Добавление LoRA адаптеров
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Степень адаптации (чем выше, тем умнее, но тяжелее)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state=42
)

/tmp/ipython-input-3727259561.py:1: UserWarning: WARNING: Unsloth should be imported before [transformers, peft] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2026.2.1 patched 32 layers with 32 QKV layers, 32 O layers and 0 MLP layers.


In [ ]:
from datasets import Dataset

# Формирование промптов для дообучения

prompt_style = """Ниже представлен план лечения. Исправь его, опираясь на клинические рекомендации.

### План лечения:
{}

### Рекомендации:
{}

### Исправленния:
{}"""

def formatting_prompts_func(examples):
    inputs = examples["treatment_plan"]
    context = examples["clinical_guidelines"]
    outputs = examples["corrections"]
    texts = []
    for i, c, o in zip(inputs, context, outputs):
        text = prompt_style.format(i, c, o) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

dataset = Dataset.from_list(data_list_train)
dataset = dataset.map(formatting_prompts_func, batched=True)
print(dataset[0]['text'])

Map:   0%|          | 0/196 [00:00<?, ? examples/s]

Ниже представлен план лечения. Исправь его, опираясь на клинические рекомендации.

### План лечения:
Возраст: 54 года.
Диагноз: Рак тела желудка, cT4bN1M1 (метастазы в печень).
Анализы: HER2: 0 (отрицательный), MSI: MSS, PD-L1: CPS 20.
Назначение:
Показана комбинация: XELOX + Ниволумаб.

### Рекомендации:
Метастатический (1-я линия): В связи с отрицательным HER2 и высоким уровнем PD-L1 (CPS ≥ 5) показана комбинация: XELOX + Ниволумаб.

### Исправленния:
Нет коррекций.<|eot_id|>


In [ ]:
import pandas as pd

# Расчет статистик по размеру запросов

def count_tokens(example):
    return {"token_count": len(tokenizer.encode(example["text"]))}

dataset_with_counts = dataset.map(count_tokens)
df = dataset_with_counts.to_pandas()

print(f"Средняя длина: {df['token_count'].mean()}")
print(f"Максимальная длина: {df['token_count'].max()}")
print(f"95-й перцентиль (рекомендуемый max_seq_length): {df['token_count'].quantile(0.95)}")

Map:   0%|          | 0/196 [00:00<?, ? examples/s]

Средняя длина: 308.08673469387753
Максимальная длина: 911
95-й перцентиль (рекомендуемый max_seq_length): 524.25


In [ ]:
import gc

# Очистка кэша перед обучением
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Дообучение модели
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/196 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 196 | Num Epochs = 3 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 13,631,488 of 8,043,892,736 (0.17% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
1,2.182200
2,2.025900
3,2.009700
4,2.212800
5,1.937400
6,1.884300
7,1.840500
8,1.739900
9,1.735200
10,1.790300


wandb: WARNING URL not available in offline run


train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
train/grad_norm,▂▁▁▃▂▂▂▃▆▃▃▃▃▃▄▃▃▇▂▃▃▇▄▆▅▄▅▅▅▅▄▄▄▄█▄▄▄▄▅
train/learning_rate,▁▂▄▇███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▁
train/loss,█▇▇█▆▆▅▅▆▅▃▄▄▄▃▃▃▂▂▂▂▂▂▂▂▃▂▂▁▁▂▂▂▁▂▂▁▂▁▁
total_flos,6574481137213440.0
train/epoch,2.40816
train/global_step,60
train/grad_norm,0.74903
train/learning_rate,0.0
train/loss,0.9404


TrainOutput(global_step=60, training_loss=1.3059756735960641, metrics={'train_runtime': 423.017, 'train_samples_per_second': 1.135, 'train_steps_per_second': 0.142, 'total_flos': 6574481137213440.0, 'train_loss': 1.3059756735960641, 'epoch': 2.4081632653061225})

In [ ]:
# Сохранение адаптеров
model.save_pretrained("medical_lora_model")
tokenizer.save_pretrained("medical_lora_model")

('medical_lora_model/tokenizer_config.json',
 'medical_lora_model/special_tokens_map.json',
 'medical_lora_model/chat_template.jinja',
 'medical_lora_model/tokenizer.json')

# Testing

In [ ]:
import torch
from unsloth import FastLanguageModel
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "medical_lora_model",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

model_name = "intfloat/multilingual-e5-small"
model_kwargs = {'device': 'cuda'}  # поменяйте на 'cuda' если есть GPU
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

db = FAISS.load_local("russco_faiss", embeddings, allow_dangerous_deserialization=True)

def get_context(treatment_plan):
    search_query = f"query: {treatment_plan}"
    docs = db.similarity_search(search_query, k=2) # Берем 3 самых релевантных чанка

    context = "\n---\n".join([
        f"Источник ({d.metadata.get('Header 2', 'Общее')}): {d.page_content}"
        for d in docs
    ])
    return context

prompt_template = """Ниже представлен план лечения. Исправь его, опираясь на клинические рекомендации.
{}
### План лечения:
{}

### Рекомендации:
{}

### Исправленния:
"""

def correct_treatment_plan(treatment_plan, context, doctor=True):
    added_text = "Формулируй ответ простыми словами, которые будут понятны пациенту.\n"
    if doctor:
        added_text = ""
    final_prompt = prompt_template.format(added_text, treatment_plan, context)
    inputs = tokenizer([final_prompt], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
    result = tokenizer.batch_decode(outputs, skip_special_tokens = True)
    corrections = result[0][len(final_prompt):]
    return corrections

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
treatment_plan = """
Диагноз: рак молочной железы.
Для определения тактики лечения проводится комплексное исследование опухоли:
ИГХ-профиль: Оценка рецепторов к гормонам (эстрогену и прогестерону), статуса HER2 и индекса пролиферации Ki-67.
Генетическое тестирование:
Поиск мутаций BRCA1/2 (обязательно при семейной истории болезни или специфических формах рака).
Анализ PIK3CA (необходим при метастатическом гормонозависимом раке для подбора таргетной терапии).
Определение уровня PD-L1 (критично для выбора иммунотерапии при тройном негативном подтипе).
Алгоритмы медикаментозного лечения
1. Профилактическое (адъювантное) лечение:
HER2-положительный тип: Стандарт — трастузумаб в течение года. При поражении лимфоузлов добавляется пертузумаб. Если после предоперационной химии осталась опухоль, назначается Кадсила (T-DM1).
Тройной негативный тип: При остаточной опухоли применяется капецитабин. Пациентам с мутацией BRCA показан олапариб.
2. Терапия при распространенных (метастатических) формах:
Гормонозависимый HER2-отрицательный: Основной стандарт первой линии — комбинация ингибиторов CDK4/6 (рибо-, палбо- или абемациклиб) с гормональной терапией. При наличии мутации PIK3CA к фулвестранту добавляют алпелисиб.
HER2-положительный: Начинают с комбинации таксанов и двойной блокады (трастузумаб + пертузумаб). Во второй линии используют T-DM1, в последующих — инновационный конъюгат T-DXd.
Тройной негативный: При высоком уровне PD-L1 эффективно сочетание химиотерапии с иммунопрепаратом пембролизумабом. Носителям мутации BRCA назначаются таргетные PARP-ингибиторы (олапариб/талазопариб).
"""

context = get_context(treatment_plan)
corrections = correct_treatment_plan(treatment_plan, context)
print(corrections)

Рекомендуется назначение PARP-ингибиторов (олапариб/талазопариб) при BRCA1/2-ассоциированных метастатических РМЖ, получавших ранее ХТ антрациклинами и таксанами.


In [ ]:
context

'Источник (4. НАБЛЮДЕНИЕ): Больным BRCA1 /2-ассоциированным HER2-негативным метастатическим РМЖ, получавшим ранее ХТ антрациклинами и таксанами с нео-/адъювантной и/или лечебной целью, рекомендуется назначение PARP-ингибиторов (табл. 11). У больных BRCA-ассоциированным гормонозависимым РМЖ оптимальная последовательность назначения PARP-ингибиторов и ГТ (± ингибиторы CDK4/6) не определена; с учетом известного увеличения продолжительности жизни при назначении ГТ в сочетании с ингибиторами CDK4/6 рекомендуется эту опцию\n---\nИсточник (**3.1. Лечение неметастатического рака поджелудочной железы (любая Т, любая N, M0)**): Единого протокола предоперационной/индукционной терапии не существует. Основным методом лечения остается ХТ. Режимы ХТ аналогичны таковым для лечения метастатического рака (табл. 4). Предпочтительными режимами (при отсутствии противопоказаний) являются (m)FOLFIRINOX или комбинации гемцитабина и nabпаклитаксела (табл. 4). При наличии у пациентов мутаций в генах BRCA или PA